In [5]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime

from networkx.classes.function import path_weight

In [6]:
def create_graph(transports):
    graph = {}

    # Creating the graph with the list of transports
    for i,transport in tqdm(transports.iterrows()):
        id_emp_orig = transport['node_src']
        id_emp_dest = transport['node_dest']

        volume = transport['vol']

        # Graph is a dict (source) -> (destination, weight)
        if graph.get(id_emp_orig) is None:
            # if source is not in the graph we need to map it as source to the destination
            graph[id_emp_orig] = {id_emp_dest: {'weight': volume }}
        else:
            # if source is already in the graph
            # 1. new destination from that source: create the edge
            # 2. source already mapped to destination: increase the volume in that edge
            if graph[id_emp_orig].get(id_emp_dest) is None:
                graph[id_emp_orig][id_emp_dest] = {'weight': volume }
            else:
                graph[id_emp_orig][id_emp_dest]['weight'] += volume

    # In this context we want to maximize paths over the volume
    # Since most functions minimizes over the weight of the edges,
    # we need to invert the relation
    for source, targets in graph.items():
        for target, volume in targets.items():
            graph[source][target]['weight'] = -1 * volume['weight']

    return nx.DiGraph(graph)

In [7]:
# Reading the transports
transports = pd.read_csv('./transports.csv')

In [8]:
# reading nodes and creating a dict for easy use
emps = pd.read_csv('./nodes.csv')

emp_type = {}

for i,node in emps.iterrows():
  emp_type[node['id_emp']] = node['type']

In [9]:
# Creating the graph
graph = create_graph(transports)
print(graph)

614253it [00:22, 27199.72it/s]


DiGraph with 51769 nodes and 82997 edges


In [10]:
G_orig_weight = graph.copy()
# nx.draw(G_orig_weight, with_labels=True, node_color='lightblue', font_weight='bold')
# plt.show()

In [11]:
G_orig_weight = graph.copy()
# nx.draw(G_orig_weight, with_labels=True, node_color='lightblue', font_weight='bold')
# plt.show()

In [12]:
def get_concessions(list_nodes,emp_type):
  # Concessions are all emps which are marked as MANEJO,
  # (legal source and extractors of timber)

  count = 0
  for node in list_nodes:
    if emp_type[node] == 'MANEJO':
      count+=1

  return count

In [13]:
def print_graph_metrics(graph, emp_type):
  # Print overall graph metrics

  g_aux = G_orig_weight.to_undirected()

  # Number of connected components in the graph
  components_len = []
  for item in nx.connected_components(g_aux):
    components_len.append((len(item),item))

  print(f'Total Graph: {graph}')
  print(f'Number of components: {len(components_len)}')
  print(f'Number of concessions: {get_concessions(graph.nodes(),emp_type)}')

  print()
  print()
  print('Components with more than 1000 nodes')
  for c in components_len:
    if c[0] > 1000:
      subg = graph.subgraph(c[1])

      print(f'Subgraph: {subg}')
      print(f'Number of concessions: {get_concessions(subg.nodes(),emp_type)}')
      print()
      print()

In [14]:
print_graph_metrics(G_orig_weight, emp_type)

Total Graph: DiGraph with 51769 nodes and 82997 edges
Number of components: 951
Number of concessions: 5026


Components with more than 1000 nodes
Subgraph: DiGraph with 48984 nodes and 80817 edges
Number of concessions: 4550




In [15]:
print(f'Biggest component proportion: {round(48984  / 51769,2) * 100}%', )

Biggest component proportion: 95.0%


This means we have one big components, with about 95% of all nodes of the graph. The remainig components is relative smaller (with less than 1000 nodes)

In [16]:
def get_sink_nodes(graph, emp_type):
  # If node is marked as FINAL, he is a sink
  # It is the final destination of the timber chain

  nodes = {}
  for node in graph.nodes():
    if emp_type[node] == 'FINAL':
      nodes[node] = 1
      continue

    # we consider sink nodes as final nodes or nodes that only transports to other final nodes
    not_sink = False
    for edge in graph.edges(node):
      if emp_type[edge[1]] != 'FINAL':
        not_sink=True

    if not not_sink:
      nodes[node] = 1

  return nodes

In [17]:
def get_timberflow(graph,emp_type):
  sink_nodes = get_sink_nodes(graph, emp_type)

  total_in = 0
  total_out = 0

  # For all edges in the graph
  # 1. sum the volume of input (from MANEJO types)
  # 2. sum the volume of output (from FINAL types)
  for edge in graph.edges():
    if emp_type[edge[0]] == 'MANEJO':
      total_in += -path_weight(graph, [edge[0],edge[1]], weight='weight')
    elif edge[1] in sink_nodes:
      total_out += -path_weight(graph, [edge[0],edge[1]], weight='weight')

  print(f'Inflow Vol(m3): {total_in} \nOut Vol(m3): {total_out} \nProportion: {total_out/total_in}')

In [18]:
# We get a higher proportion because of the missing data from Mato Grosso
get_timberflow(G_orig_weight, emp_type)

Inflow Vol(m3): 1969118.5468748265 
Out Vol(m3): 1252975.495200041 
Proportion: 0.6363128808007162


Here we have a higher proportion of out/in because MT has one of the biggest inflows.

In [19]:
def dijkstra(G, source, emp_type, k):
    # Compute the possible paths from source to MANEJO origin ordered by weight
    # Using dijkstra, we assign more probability of path existance maximized by volume flow
    lenght, paths = nx.single_source_dijkstra(G, source)
    target_weights = {}

    for target, path in paths.items():
        if emp_type[target] == 'MANEJO':
            target_weights[target] = path_weight(G, path, weight='weight')

    # List of paths from source to MANEJO ordered by weight
    target_ordered = sorted(target_weights.items(), key = lambda x: x[1])[:k]

    return paths, target_ordered


def eppistein(G, source, target, k):
    paths = []
    for path in nx.shortest_simple_paths(G, source, target, weight='weight'):
        # If we already have K paths, or if current path is greather than next
        # dijkstra path, we can skip this one.

        if len(paths) >= k:
            break
        # Else, we put this path into the list
        paths.append((path, path_weight(G, path, weight='weight')))

    return paths


def eppistein_graph(G, source, target_weight, k, dijkstra_paths):
    # Return the K shortest paths to reach MANEJO from source node
    paths = []

    for i, (target, weight) in enumerate(target_weight):
        paths += eppistein(G, source, target, k)

    limit = k if len(paths) >= k else len(paths)
    return sorted(paths, key=lambda x: x[1])[:limit]


def original_path_cost(paths, original_graph):
    # Assigning the correct path from the original_graph
    for i, (path, weight) in enumerate(paths):
        paths[i] = (path, -1.0 * path_weight(original_graph, path, weight='weight'))

    return paths

In [20]:
# Graph is a directed graph from source to final nodes
# Since we want to find the path from final nodes to source, we need to reverse it
graph_reversed = graph.reverse().copy()
G = graph_reversed.copy()

# Trick thing here:
# Dijkstra works with positive values as weights, so to achieve the maximun path
# we need to multiply all weights by -1 and subtract all weights from the minimun

# This way, the maximun weighted edge ih the original graph will have weight 0
# and we can perform the shortest path algorithms
min_weight = min(G.edges(data='weight'), key=lambda x: x[2])[2]

attrs = {}
for i, edge in enumerate(G.edges):
    attrs[edge] = G.edges[edge]
    attrs[edge]['weight'] += -min_weight

nx.set_edge_attributes(G, attrs)

In [21]:
# paths
k = 3

# Dijkstra will encounter a path from a node to each other node with minimun path cost
# We want the K most probable paths from source to any other node, where this node can repeat
# Therefore, we cannot use only dijkstra for this, we need to check from eppistein algoritm to find
# the alternative paths

# Dijkstra will find a path from SOURCE to NODE_A, NODE_B, ..., each one minimized for each tuple (SOURCE, DESTINATION)
# Eppistein will find the paths from SOURCE to NODE_A ordered by the path weight
# Therefore, if we have a second path from SOURCE to NODE_A with a better weight than SOURCE to NODE_B (from dijkstra)
# we will uprank A path and downrank the B path.

# This is the algoritm to find the K paths.
paths, target_ordered = dijkstra(G, 518879, emp_type, k)
secondary_paths = eppistein_graph(G, 518879, target_ordered, k, paths)

# Paths with weights used for the algorithm
secondary_paths

[([518879, 518628], 19753.938500000015),
 ([518879, 1194995, 518841], 39379.79040000003),
 ([518879, 1194995, 518628], 39405.01040000003)]

In [22]:
# Paths with corrected weights
secondary_paths = original_path_cost(secondary_paths, graph_reversed)

for path, weight in secondary_paths:
  print(f"(FINAL) {' <- '.join(map(str, path))} (SOURCE) - Weight: {round(weight, 2)} M3")

(FINAL) 518879 <- 518628 (SOURCE) - Weight: 5.3 M3
(FINAL) 518879 <- 1194995 <- 518841 (SOURCE) - Weight: 138.69 M3
(FINAL) 518879 <- 1194995 <- 518628 (SOURCE) - Weight: 113.47 M3


In [23]:
emps = pd.read_csv('./nodes.csv')

In [24]:
emps.head()

,id_emp,latitude,longitude,type
0,2612348,-23.5329,-46.6395,FINAL
1,2612495,-20.8169,-49.5206,FINAL
2,2613149,-20.5352,-47.4039,FINAL
3,2613315,-23.4439,-46.9178,FINAL
4,2613344,-23.5329,-46.6395,FINAL


In [25]:
emps_filtered = emps[emps['id_emp'].isin(set([node for (path, weight) in secondary_paths for node in path]))].copy()
emps_filtered['size'] = 10

In [26]:
emps_filtered

,id_emp,latitude,longitude,type,size
35079,518628,-20.1597,-56.6600,MANEJO,10
35093,518841,-20.1497,-56.6300,MANEJO,10
35096,518879,-20.2355,-56.3746,PTO_IBAMA,10
38985,1194995,-20.1597,-56.6600,PTO_IBAMA,10


In [28]:
import plotly.graph_objects as go
import plotly.express as px

for path, weight in secondary_paths:
  fig = px.scatter_mapbox(
      emps_filtered[emps_filtered['id_emp'].isin(path)],
      lat="latitude",
      lon="longitude",
      zoom=8,
      mapbox_style='open-street-map',
      color='type',
      size='size',
      size_max=15,
      width=1000,
      height=500)

  fig.show()